In [1]:
import os
import argparse
from filelock import FileLock
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest import ConcurrencyLimiter
import webdataset as wds
import random
import numpy as np
from typing import Type, Any, Callable, Union, List, Optional
from torch import Tensor


PATH_TO_DATA =  "/mnt/analysis/analysis/rand_sharded_data/" 

EPOCHS = 50
BATCH_SIZE = 512

In [2]:
image_normalize = transforms.Normalize(
                  mean=[0, 0, 0, 0, 4.0999e+02],
                  std=[1, 1, 1, 1, 7.4237e+01]
)
#with clipping to 600: 4.0999e+02, 7.4237e+01
#without clipping: 4.1454e+02, 8.9422e+01

forcing_normalize = transforms.Normalize(
                  mean=[444.9605606256559, 991.7980623653417, 0.00039606951184754176, 96111.04161525163, 0.006652783216819315, 314.3219695851273, 2.82168247768119],
                  std=[5.5216369223813535, 12.951212256256913, 0.0002824274832735609, 975.3770569179914, 0.00012386107613000674, 0.6004463118907452, 0.34279194598853185]
)

elevation_mean = torch.from_numpy(np.array([4.0999e+02]))
elevation_std = torch.from_numpy(np.array([7.4237e+01]))

forcing_mean = torch.from_numpy(np.array([444.9605606256559, 991.7980623653417, 0.00039606951184754176, 96111.04161525163, 0.006652783216819315, 314.3219695851273, 2.82168247768119]))
forcing_std = torch.from_numpy(np.array([5.5216369223813535, 12.951212256256913, 0.0002824274832735609, 975.3770569179914, 0.00012386107613000674, 0.6004463118907452, 0.34279194598853185]))

lst_mean = torch.from_numpy(np.array([312.8291360088677]))
lst_std = torch.from_numpy(np.array([11.376636496297289]))

In [3]:
def create_train_test(path_to_data, train_perc, test_perc):
    files = []
    for dirpath, dirnames, filenames in os.walk(path_to_data):
        files.extend(filenames)
    
    saturated = files[:-1]
    unsaturated = files[-1]
    
    dataset = wds.WebDataset(path_to_data + "/" + unsaturated)
    counter = 0
    for data in dataset:
        counter += 1
    
    total_files = counter + len(saturated) * 10000
    training_data = total_files * train_perc //10000
    test_data_files = total_files * test_perc //10000

    training_data = random.sample(files, int(training_data))
    test_data = [file for file in files if file not in training_data]
    test_data = random.sample(test_data, int(test_data_files))
    # Get sample sizes of train and test data
    training_samples = 0
    testing_samples = 0
    
    for path in training_data:
        if path in saturated:
            training_samples += 10000
        elif path in unsaturated:
            training_samples += counter
            
    for path in test_data:
        if path in saturated:
            testing_samples += 10000
        elif path in unsaturated:
            testing_samples += counter
            
            
    # Convert to filename lists 
    training_filepath = []
    for dat in training_data:
        training_filepath.append(dat[6:12])
    training_path = path_to_data + "shard-" + "{" + ",".join(training_filepath) + "}" + ".tar"
    
    testing_filepath = []
    for dat in test_data:
        testing_filepath.append(dat[6:12])
    testing_path = path_to_data + "shard-{" + ",".join(testing_filepath) +"}.tar"
    train_data = wds.WebDataset(training_path).shuffle(30000, initial=30000).decode("rgb").rename(image="image.pyd", forcing="forcing.pyd", lst = "lst.pyd").to_tuple("image", "forcing", "lst")
    test_data = wds.WebDataset(testing_path).decode("rgb").shuffle(30000, initial=30000).rename(image="image.pyd", forcing="forcing.pyd", lst = "lst.pyd").to_tuple("image", "forcing", "lst")
    
    all_filepath = []
    for dat in saturated:
        all_filepath.append(dat[6:12])
    all_path = path_to_data + "shard-" + "{" + ",".join(all_filepath) + "}" + ".tar"
    
    all_data = wds.WebDataset(all_path).decode("rgb").rename(image="image.pyd", forcing="forcing.pyd", lst = "lst.pyd").to_tuple("image", "forcing", "lst")
   
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE)

    return (train_loader, training_samples), (test_loader, testing_samples), all_data
    
(train_loader, training_samples_len), (test_loader, testing_samples_len), all_data = create_train_test(PATH_TO_DATA, 0.9, 0.1)

In [4]:

def process_data(image, forcing, lst, DEVICE):
    image, forcing, lst = image.to(DEVICE).to(torch.float32), forcing.to(DEVICE), lst.to(DEVICE)
    # Image Transformations
    image[:,4,] = torch.clip(image[:,4,], min=0, max=600)
    image[:,:4,] = torch.clip(image[:,:4,], min=0, max=1)
    image = image_normalize(image)
    # Forcing Transformation
    forcing = torch.div(torch.sub(forcing, forcing_mean), forcing_std).to(torch.float32)
    # LST Transformation
    lst = lst.view(-1, 1).to(torch.float32)
    return image, forcing, lst


def conv3x3(in_planes: int, out_planes: int, stride: int = 1, groups: int = 1, dilation: int = 1) -> nn.Conv2d:
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes: int, out_planes: int, stride: int = 1) -> nn.Conv2d:
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion: int = 1

    def __init__(
        self,
        inplanes: int,
        planes: int,
        stride: int = 1,
        downsample: Optional[nn.Module] = None,
        groups: int = 1,
        base_width: int = 64,
        dilation: int = 1,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x: Tensor) -> Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = F.leaky_relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = F.leaky_relu(out)

        return out

class ResNet(nn.Module):
    def __init__(
        self,
        block=BasicBlock,
        layer_1_num=3,
        layer_2_num=3,
        layer_3_num=0,
        layer_4_num=0,
        linear_1 = 64,
        linear_2 = 32,
        drop_1_p = 0.2,
        drop_2_p = 0.2,
        in_channel=5,
        forcing_shape = 7,
        zero_init_residual: bool = False,
        groups: int = 1,
        width_per_group: int = 64,
        replace_stride_with_dilation: Optional[List[bool]] = None,
        norm_layer: Optional[Callable[..., nn.Module]] = None,
    ) -> None:
        super().__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                f"or a 3-element tuple, got {replace_stride_with_dilation}"
            )
        self.groups = groups
        self.base_width = width_per_group
        self.flatten_shape = None
        
        self.layer_1_num = layer_1_num
        self.layer_2_num = layer_2_num
        self.layer_3_num = layer_3_num
        self.layer_4_num = layer_4_num
        
        self.conv1 = nn.Conv2d(in_channel, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.avgpool1 = nn.AvgPool2d(2)
        self.layer1 = self._make_layer(block, 64, layer_1_num)
        self.layer2 = self._make_layer(block, 128, layer_2_num, stride=2, dilate=replace_stride_with_dilation[0])
        if self.layer_3_num != 0:
            self.layer3 = self._make_layer(block, 256, layer_3_num, stride=2, dilate=replace_stride_with_dilation[1])
        if self.layer_4_num != 0:
            self.layer4 = self._make_layer(block, 512, layer_4_num, stride=2, dilate=replace_stride_with_dilation[2])
        self.avgpool2 = nn.AdaptiveAvgPool2d((1,1))
        
        self.linear_shape = None
        if self.linear_shape is None:
            randn = torch.randn((12, 5, 33, 33))
            with torch.no_grad():
                out = self.convolutions(randn)
                self.linear_shape = out.shape[1] *out.shape[2]* out.shape[3] + 7
        self.fc0 = nn.Linear(self.linear_shape, out_features=linear_1)
        self.drop1 = nn.Dropout(drop_1_p)
        self.fc1 = nn.Linear(linear_1, linear_2)
        self.drop2 = nn.Dropout(drop_2_p)
        self.fc2 = nn.Linear(linear_2, out_features=1)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)  # type: ignore[arg-type]
    
    def _make_layer(
        self,
        block,
        planes: int,
        blocks: int,
        stride: int = 1,
        dilate: bool = False,
    ) -> nn.Sequential:
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = nn.ModuleList([])
        layers.append(
            block(
                self.inplanes, planes, stride, downsample, self.groups, self.base_width, previous_dilation, norm_layer
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)
    
    def convolutions(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.leaky_relu(x)
        x = self.avgpool1(x)

        x = self.layer1(x)
        x = self.layer2(x)
        if self.layer_3_num != 0:
            x = self.layer3(x)
        if self.layer_4_num != 0:
            x = self.layer4(x)
        x = self.avgpool2(x)
        return x
        
    def forward(self, x: Tensor, forcing) -> Tensor:
        # See note [TorchScript super()]
        x = self.convolutions(x)
        x = torch.flatten(x, 1)
        x = torch.cat((x, forcing), dim=1)
        x = self.fc0(x)
        x = F.leaky_relu(x)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.drop1(x)
        x = self.fc2(x)

        return x
## OPTIMIZE PARAMS ###
# layer_1_num=3,
# layer_2_num=3,
# layer_3_num=0,
# layer_4_num=0,
# linear_1 = 64,
# linear_2 = 32,
# drop_1_p = 0.2,
# drop_2_p = 0.2,

In [5]:
def train_model(model, optimizer, train_loader, device=None):
    device = device or torch.device("cpu")
    model.train()
    loss_fn = nn.SmoothL1Loss()
    for epoch in range(EPOCHS):
        for image, forcing, lst in train_loader:
            image, forcing, lst = process_data(image, forcing, lst, device)
            optimizer.zero_grad()
            forward_out = model.forward(image, forcing)
            loss = loss_fn(forward_out, lst)
            loss.backward()
            optimizer.step()
            
def test_model(model, test_loader, device=None):
    device=device or torch.device("cpu")
    model.eval()
    running_loss = 0
    n_iter = 0
    loss_fn = nn.MSELoss()
    with torch.no_grad():
        for image, forcing, lst in test_loader:
            image, forcing, lst = process_data(image, forcing, lst, device)
            forward_out = model(iamge, forcing)
            loss = loss_fn(forward_out, lst)
            running_loss += loss.item()
            n_iter += 1
    return running_loss / n_iter
            
def training_script(config):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    (train_loader, _), (test_loader, _), _ = create_train_test(PATH_TO_DATA, 0.9, 0.1)
    model = ResNet(layer_1_num = config["layer_1_num"],
                   layer_2_num = config["layer_2_num"],
                   layer_3_num = config["layer_3_num"],
                   layer_4_num = config["layer_4_num"],
                   linear_1 = config["linear_1"],
                   linear_2 = config["linear_2"],
                   drop_1_p = config["drop_1_p"],
                   drop_2_p = config["drop_2_p"])
    optimizer = optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
    
    while True:
        train_model(model, optimizer, train_loader, device)
        acc = test_model(model, test_loader, device)
        tune.report(mean_accuracy=acc)
        

In [6]:
ray.init(num_cpus=1, num_gpus=2)
sched = AsyncHyperBandScheduler()
algo = HyperOptSearch()

In [7]:
config = {
    "layer_1_num": tune.choice([1, 2, 3]),
    "layer_2_num": tune.choice([1, 2, 3]),
    "layer_3_num": tune.choice([0, 1, 2, 3]),
    "layer_4_num": tune.choice([0, 1, 2, 3]),
    "linear_1": tune.randint(12, 100),
    "linear_2": tune.randint(12, 100),
    "drop_1_p": tune.uniform(0, 0.6),
    "drop_2_p": tune.uniform(0, 0.6),
    "lr": tune.uniform(1e-5, 1e-2),
    "weight_decay": tune.uniform(1e-3, 1e-2),
}

In [ ]:
analysis = tune.run(training_script,
                    local_dir=os.getcwd(),
                    name="exp",
                    metric="mean_accuracy",
                    mode="min",
                    search_alg=algo,
                    scheduler=sched,
                    resources_per_trial={
                        "cpu":1,
                        "gpu":1
                    },
                   
                    num_samples=20,
                    config = config)

2022-04-18 08:42:23,659	WARNING function_runner.py:598 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2022-04-18 08:42:23,849	INFO trial_runner.py:803 -- starting training_script_609639a0


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946


(training_script pid=64474) /home/priyammazumdar/anaconda3/envs/research/lib/python3.9/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272068185/work/c10/cuda/CUDAFunctions.cpp:112.)
(training_script pid=64474)   return torch._C._cuda_getDeviceCount() > 0


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


...

Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


Trial name,status,loc,drop_1_p,drop_2_p,layer_1_num,layer_2_num,layer_3_num,layer_4_num,linear_1,linear_2,lr,weight_decay
training_script_609639a0,RUNNING,192.168.1.108:64474,0.198334,0.0513232,1,3,3,2,64,88,0.00888175,0.00317946
training_script_61ca8ba2,PENDING,,0.440468,0.154535,2,3,1,1,57,15,0.00239941,0.00148879


In [ ]:
ray.shutdown()